In [ ]:
# Chapter 8: GENERATING IMG USING VAEs and GANs
"""The main idea is to dev a low-dim latent space of representations where any point can be mapped to a realistic-lookinng-img
   taking as io(lantent_point->output_img)      not(predict) but(generator[GANs], decoder[VAEs])
   """

Returning: png/3D

Variational autoencoders, simultaneously discovered by Kingma and Welling in
December 2013 6 and Rezende, Mohamed, and Wierstra in January 2014,7 are a kind
of generative model that’s especially appropriate for the task of image editing via con-
cept vectors.

In [ ]:
"""z_mean, z_log_variance = encoder(input_img)
# Encodes the input into a
# mean and variance parameter
z = z_mean + exp(z_log_variance) * epsilon
# Decodes z back to an image
# Draws a latent point using a small random epsilon
reconstructed_img = decoder(z)
model = Model(input_img, reconstructed_img)
# Instantiates the autoencoder model, which maps an input image to its reconstruction"""

In [ ]:
"""Listing 8.23 VAE encoder network"""
import keras
from keras import layers
from keras import backend as K
from keras.models import Model
import numpy as np
img_shape = (28, 28, 1)
batch_size = 16
latent_dim = 2

# Dimensionality of the latent space: a 2D plane
input_img = keras.Input(shape=img_shape)

# Generating images with variational autoencoders
x = layers.Conv2D(32, 3,
padding='same', activation='relu')(input_img)
x = layers.Conv2D(64, 3,
padding='same', activation='relu',
strides=(2, 2))(x)
x = layers.Conv2D(64, 3,
padding='same', activation='relu')(x)
x = layers.Conv2D(64, 3,
padding='same', activation='relu')(x)
shape_before_flattening = K.int_shape(x)
x = layers.Flatten()(x)
x = layers.Dense(32, activation='relu')(x)
z_mean = layers.Dense(latent_dim)(x)
z_log_var = layers.Dense(latent_dim)(x)

# The input image ends up being encoded into 
# these two parameters.

In [ ]:
"""Listing 8.24 Latent-space-sampling function"""
def sampling(args):
    z_mean, z_log_var = args
    epsilon = K.random_normal(shape=(K.shape(z_mean)[0], latent_dim),
    mean=0., stddev=1.)
    return z_mean + K.exp(z_log_var) * epsilon
z = layers.Lambda(sampling)([z_mean, z_log_var])

In [ ]:
# Listing 8.25 VAE decoder network, mapping latent space points to images
decoder_input = layers.Input(K.int_shape(z)[1:]) #Input where you’ll feed z
x = layers.Dense(np.prod(shape_before_flattening[1:]),
activation='relu')(decoder_input) #Upsamples the input
x = layers.Reshape(shape_before_flattening[1:])(x)
x = layers.Conv2DTranspose(32, 3,
padding='same',
activation='relu',
strides=(2, 2))(x)
x = layers.Conv2D(1, 3,
padding='same',
activation='sigmoid')(x)
"""Reshapes z into a feature map of the same shape as the feature
map just before the last Flatten layer in the encoder mode"""

"""Uses a Conv2DTranspose layer and Conv2D layer to decode z into
a feature map the same size as the original image input302 """
decoder = Model(decoder_input, x)
z_decoded = decoder(z)
#Applies it to z to recover the decoded z

In [ ]:
# Listing 8.26 Custom layer used to compute the VAE loss
class CustomVariationalLayer(keras.layers.Layer):
    def vae_loss(self, x, z_decoded):
        x = K.flatten(x)
        z_decoded = K.flatten(z_decoded)
        xent_loss = keras.metrics.binary_crossentropy(x, z_decoded)
        kl_loss = -5e-4 * K.mean(
        1 + z_log_var - K.square(z_mean) - K.exp(z_log_var), axis=-1)
        return K.mean(xent_loss + kl_loss)
    
    """You don't use this output, but the layer must returnsomething."""
    def call(self, inputs):
        #You implement custom layers by writing a call method.
        x = inputs[0]
        z_decoded = inputs[1]
        loss = self.vae_loss(x, z_decoded)
        # Calls the custom layer on the input and the
        self.add_loss(loss, inputs=inputs)
        return x


# decoded output to obtain the final model output
y = CustomVariationalLayer()([input_img, z_decoded])

In [ ]:
"""Listing 8.27 Training the VAE"""
from keras.datasets import mnist
vae = Model(input_img, y)
vae.compile(optimizer='rmsprop', loss=None) 
 vae.summary()
(x_train, _), (x_test, y_test) = mnist.load_data()
x_train = x_train.astype('float32') / 255.
x_train = x_train.reshape(x_train.shape + (1,))
x_test = x_test.astype('float32') / 255.
x_test = x_test.reshape(x_test.shape + (1,))
vae.fit(x=x_train, y=None,
    shuffle=True,
    epochs=10,
    batch_size=batch_size,
    validation_data=(x_test, None)
)

In [ ]:
"""Listing 8.28 Sampling a grid of points from the 2D latent space 
   and decoding them to images"""
from scipy.stats import norm
import matplotlib.pyplot as plt


# You’ll display a grid of 15 × 15 digits (255 digits total).
n, digit_size = 15, 28
figure = np.zeros((digit_size * n, digit_size * n))
grid_x = norm.ppf(np.linspace(0.05, 0.95, n))
grid_y = norm.ppf(np.linspace(0.05, 0.95, n))
"""Transforms linearly spaced coordinates using the SciPy ppf
    function to produce values of the latent variable z
    (because the prior of the latent space is Gaussian)"""

#Repeats z multiple times to form a complete batch
for i, yi in enumerate(grid_x):
    for j, xi in enumerate(grid_y):
        z_sample = np.array([[xi, yi]])
        z_sample = np.tile(z_sample, batch_size).reshape(batch_size, 2)
        x_decoded = decoder.predict(z_sample, batch_size=batch_size)
        digit = x_decoded[0].reshape(digit_size, digit_size)
        figure[i * digit_size: (i + 1) * digit_size,
        j * digit_size: (j + 1) * digit_size] = digit

plt.figure(figsize=(10, 10))
plt.imshow(figure, cmap='Greys_r')
plt.show()
"""Reshapes the first digit in the batch
    from 28 × 28 × 1 to 28 × 28
    Decodes the batch into digit images"""